In [1]:
import re
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
# Import your data to a Pandas.DataFrame
df = pd.read_excel('/Users/maustin/Desktop/WBL Data Request 20220310[56].xlsx', sheet_name = 'OrgNames')

# Grab the column you'd like to group, filter out duplicate values
# and make sure the values are Unicode
vals = df['OrgName'].unique().astype('U')


# Write a function for cleaning strings and returning an array of ngrams
def ngrams_analyzer(string):
    string = re.sub(r'[,-./]', r'', string)
    ngrams = zip(*[string[i:] for i in range(5)])  # N-Gram length is 5
    return [''.join(ngram) for ngram in ngrams]

# Construct your vectorizer for building the TF-IDF matrix
vectorizer = TfidfVectorizer(analyzer=ngrams_analyzer)

# Build the matrix!!!
tfidf_matrix = vectorizer.fit_transform(vals)

In [3]:
# !pip install cython
# !pip install git+https://github.com/ing-bank/sparse_dot_topn.git

In [4]:
# !pip install --upgrade numpy

# !pip uninstall pycocotools
# !pip install pycocotools --no-binary pycocotools

# !pip uninstall pycocotools-y
# !pip install --no-cache-dir pycocotools

In [5]:
# Import IGN's awesome_cossim_topn module
from sparse_dot_topn import awesome_cossim_topn as ct

# The arguments for awesome_cossim_topn are as follows:
### 1. Our TF-IDF matrix
### 2. Our TF-IDF matrix transposed (allowing us to build a pairwise cosine matrix)
### 3. A top_n filter, which allows us to filter the number of matches returned, which isn't useful for our purposes
### 4. This is our similarity threshold. Only values over 0.8 will be returned
cosine_matrix = ct(
  tfidf_matrix,
  tfidf_matrix.transpose(),
  vals.size,
  0.7
)

In [6]:
# Build a coordinate matrix from a cosine matrix
coo_matrix = cosine_matrix.tocoo()

# Instaniate our lookup hash table
group_lookup = {}


def find_group(row, col):
    # If either the row or the col string have already been given
    # a group, return that group. Otherwise return none
    if row in group_lookup:
        return group_lookup[row]
    elif col in group_lookup:
        return group_lookup[col]
    else:
        return None


def add_vals_to_lookup(group, row, col):
    # Once we know the group name, set it as the value
    # for both strings in the group_lookup
    group_lookup[row] = group
    group_lookup[col] = group


def add_pair_to_lookup(row, col):
    # in this function we'll add both the row and the col to the lookup
    group = find_group(row, col)  # first, see if one has already been added
    if group is not None:
        # if we already know the group, make sure both row and col are in lookup
        add_vals_to_lookup(group, row, col)
    else:
        # if we get here, we need to add a new group.
        # The name is arbitrary, so just make it the row
        add_vals_to_lookup(row, row, col)

# for each row and column in coo_matrix
# if they're not the same string add them to the group lookup
for row, col in zip(coo_matrix.row, coo_matrix.col):
    if row != col:
        # Note that what is passed to add_pair_to_lookup is the string at each index
        # (eg: the names in the legal_name column) not the indices themselves
        add_pair_to_lookup(vals[row], vals[col])

In [7]:
df['Group'] = df['OrgName'].map(group_lookup).fillna(df['OrgName'])

df.to_csv('/Users/maustin/Desktop/GWC_Grouped_OrgNames.csv')